In [3]:
# En caso de problemas, utilizar las dependencias de librerías de este requierement: https://github.com/googlecolab/backend-info/blob/d6d345cb94fc5fd49951c9af0f6ead5e962bfab2/pip-freeze.txt
# !pip install numpy==1.23.5
# !pip install transformers[torch]==4.35.2
# !pip install accelerate -U
# !pip install evaluate

In [4]:
import pandas as pd
def load_prepare_data(path):
  """
  Función para cargar y procesar datos para el ejercicio.
  """
  df = pd.read_csv(path,sep=",")
  map_classes = {
    "religion":1,
    "age":1,
    "ethnicity":1,
    "gender":1,
    "other_cyberbullying":1,
    "not_cyberbullying":0,
  }
  df["cyberbullying"] = df.cyberbullying_type.map(map_classes)
  return df[["tweet_text","cyberbullying"]].copy()

# Ejercicio


En este ejercicio vamos a trabajar con un conjunto de datos procedente de medios sociales online.

Uno de los mayores problemas en el internet de hoy en día es la presencia de actitudes negativas hacia algunos colectivos en relación a su etnia, género, religión o ideología política. En este ejercicio trabajaremos con un conjunto de datos reales, etiquetados manualmente, procedentes de la plataforma [Kaggle](https://www.kaggle.com/datasets/andrewmvd/cyberbullying-classification/data). Originalmente, a cada documento del dataset se le asignó una de las siguientes categorías:
- *religion*
- *age*
- *ethnicity*
- *gender*
- *other_cyberbullying*
- *not_cyberbullying*


El objetivo inicial del dataset era su uso para entrenar un modelo capaz de detectar el tipo de contenido de odio presente en internet según el colectivo al que se atacaba. En este caso, para simplificar el ejercicio, se ha generado una función `load_prepare_data()` que cambia las categorías del dataset obteníendose al final 2 categorías con valor 1 o 0, indicando si el tweet tiene contenido de odio

**En este ejercicio debeis entrenar un modelo de clasificación utilizando la librería Transformers.** Dado que el análisis exploratorio ha sido realizado en el ejercicio anterior, en este caso podréis centraros en entrenar el modelo utilizando la librería Transformers, seleccionando un modelo pre-entrenado adecuado, entrenando el modelo y llevando a cabo la evaluación.


**Nota 1**: Este ejercicio requiere el uso de las GPUs de Google Colab. Este Colab debería estar preconfigurado para ejecutarse en GPU, pero si tuviera problemas en la ejecución que me contacte a través del Moodle para buscar soluciones alternativas.

## 0. Imports


In [5]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   AutoModelForSequenceClassification,
   AdamW
)
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

## 1. Obtención del corpus
Para la obtención de los datos teneis disponible la función `load_prepare_data()`. Esta función prepara los datos del ejercicio en formato Pandas dataframe para que podais realizarlo.

In [6]:
path_data = "https://raw.githubusercontent.com/luisgasco/ntic_master_datos/main/datasets/cyberbullying_tweets.csv"
# Path de datos alternativos en caso de que el anterior no funcione (al estar alojado en github puede haber limitaciones
# en la descarga.
# path_data = "https://zenodo.org/records/10938455/files/cyberbullying_tweets.csv?download=1"
dataset = load_prepare_data(path_data)
#en esta celda de código hemos ejecutado la funcion loaad_prepare_data()

In [7]:
dataset.head(7)

,tweet_text,cyberbullying
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0
2,@XochitlSuckkks a classy whore? Or more red ve...,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0
4,@RudhoeEnglish This is an ISIS account pretend...,0
5,"@Raja5aab @Quickieleaks Yes, the test of god i...",0
6,Itu sekolah ya bukan tempat bully! Ga jauh kay...,0


## 2. Análisis exploratorio

Podéis saltarlo en este ejercicio.

## 3. Preprocesado y Normalización

para la realización del preprocesado realizaremos una division del dataset separando las etiquetas del resto de variables del conjunto.


In [8]:
textos = dataset.tweet_text.values
etiquetas = dataset.cyberbullying.values

Una vez divididas las variables del modelo, divido la muestra en dos (80% para train y 20%  test)

In [9]:
train_texts, test_texts, train_labels, test_labels = train_test_split(textos, etiquetas, test_size=.2, random_state=0,
                                                    stratify = etiquetas)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=0,stratify = train_labels)

Para generar un modelo de clasificación con la librería Transformer primero hay que entrenar un pre-modelo.
Además, cada modelo de preentreno en la librería Transformer tiene un tokenizador específico para optmizarlo, por lo que hay que elegir el modelo y procesar los datos de acuerdo a ello. En mi caso voy a elegir el modelo bert-base-uncased

In [10]:
model_name = 'bert-base-uncased'
tokenizador = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## 4. Vectorización


Con las siguientes funciones desarrolladas en clase trabajaremos sobre el modelo por lo que hay que transformar primeramente los datos a una estructura que sea entendible por las funciones y posteriormente por el modelo.

In [11]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        """
        Constructor de la clase CustomDataset.
        Parámetros:
        - texts: Lista de textos.
        - labels: Lista de etiquetas correspondientes a los textos.
        - tokenizer: Objeto del tokenizador a utilizar.
        - max_length: Longitud máxima de la secuencia después de la tokenización.
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        """
        Devuelve la longitud del conjunto de datos.
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Obtiene un elemento del conjunto de datos.

        Parámetros:
        - idx: Índice del elemento a obtener.

        Devuelve:
        Un diccionario con 'input_ids', 'attention_mask' y 'labels'.
        """
        # Obtener el texto y la etiqueta del índice proporcionado
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        # Tokenizar el texto
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        # Devolver el diccionario con los datos
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [12]:
max_length = 280
#maxima longitud de caracteres en tweeter
train_dataset = CustomDataset(train_texts, train_labels, tokenizador, max_length)
val_dataset = CustomDataset(val_texts, val_labels, tokenizador, max_length)
test_dataset = CustomDataset(test_texts, test_labels, tokenizador, max_length)

Antes de entrenar el modelo, hay que definir tanto la longitud de la secuencia como el Batch size

In [13]:
max_seq_length = 96 # @param {"type":"integer","placeholder":"100"}
train_batch_size =  8#@param {type: "integer"}
eval_batch_size = 8 #@param {type: "integer"}
test_batch_size = 8 #@param {type: "integer"}

## 5. Entrenamiento y evaluación de modelos


Para poder ejecutar el entrenamiento hay que cargar el modelo en el entorno de ejecucción. También le voy a especificar el número de etiquetas que tengo en el modelo (2).

Además, en la siguiente caja de código,  voy a definir parámetros para poder ejecutar el entrenamiento.

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "NO_CYBERBULLYING", 1: "CYBERBULLYING"}
label2id = {"NO_CYBERBULLYING": 0, "CYBERBULLYING": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_name,  num_labels=2, id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
import accelerate

training_args = TrainingArguments(
    output_dir="modelo_test",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Mediante esta función voy a poder calculas las métricas de evaluación del modelo que voy a crear.

In [23]:
import numpy as np
!pip install evaluate
import evaluate

accuracy = evaluate.load("accuracy")
f1_score = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy_value = accuracy.compute(predictions=predictions, references=labels)
    f1_score_value = f1_score.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_value,
        "f1_score": f1_score_value,
    }


Creo el objeto trainer el cual me va a permitir ajustar el modelo y posteriormente lo entreno.

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizador,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.258600,0.265325,{'accuracy': 0.8909710391822828},{'f1': 0.9367204137511409}
2,0.205000,0.372785,{'accuracy': 0.8895295505176255},{'f1': 0.9352385342244757}


Trainer is attempting to log a value of "{'accuracy': 0.8909710391822828}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9367204137511409}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.8895295505176255}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9352385342244757}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=7632, training_loss=0.22391887380891876, metrics={'train_runtime': 3448.7984, 'train_samples_per_second': 17.7, 'train_steps_per_second': 2.213, 'total_flos': 8783551472169600.0, 'train_loss': 0.22391887380891876, 'epoch': 2.0})

El modelo muestra buen rendimiento general, con una alta exactitud y F1 score desde la primera época. Sin embargo, el aumento en la pérdida de validación durante la segunda época sugiere que podría estar ocurriendo sobreajuste. Se recomienda monitorear este comportamiento y considerar ajustes como la regularización o detener el entrenamiento antes de que el modelo comience a perder generalización.


In [29]:
predicciones = trainer.evaluate(test_dataset)

KeyboardInterrupt: 

El modelo tiene un buen rendimiento en el subconjunto de prueba, con una exactitud del 89% y un F1 score de 0.936, lo que sugiere que es capaz de generalizar bien en datos no vistos. Sin embargo, hay un ligero aumento en la pérdida en comparación con la fase de entrenamiento, lo que puede indicar una pequeña tendencia al sobreajuste. En general, los resultados son positivos y coherentes con los obtenidos durante la fase de validación.

Método .evaluate()

Classification report

Primero me guardo las predicciones de mi modelo y los valores reales de mis datos

In [32]:
y_pred = predicciones.argmax(axis=1)

KeyboardInterrupt: 

In [ ]:
y_true = [x["labels"].item() for x in test_dataset]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))